In [ ]:
from langchain_community.llms import Ollama
from langchain.agents import tool
from langchain.agents import tool, initialize_agent, AgentType, Tool
import sys
import os


sys.path.append('../../bluesky')  # Adjust the path as necessary


# Now you can import bluesky modules

sys.path.append(os.path.abspath('../'))
import os
import streamlit as st

import bluesky

from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

from langchain.agents.format_scratchpad.openai_tools import (

    format_to_openai_tool_messages,

)


from langchain_community.tools.tavily_search import TavilySearchResults

from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import ChatOpenAI

from openai import OpenAI

from io import StringIO

from contextlib import contextmanager
import time

from dotenv import load_dotenv, find_dotenv

import chromadb.utils.embedding_functions as embedding_functions

import chromadb

from bluesky.network.client import Client

from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper

from prompts.prompts import conflict_prompt


from langchain.output_parsers import ResponseSchema, StructuredOutputParser

from langchain_experimental.utilities import PythonREPL

from langchain.agents import Tool

In [ ]:
# Load the .env file
load_dotenv(find_dotenv())

In [ ]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-large"
)
vectordb_path = 'C:/Users/justa/OneDrive/Desktop/Developer/LLM-Enhanced-ATM/llm/skills-library/vectordb'
chroma_client = chromadb.PersistentClient(path=vectordb_path)
# Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection = chroma_client.get_or_create_collection(
    name="test2", embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})
collection.peek()

In [ ]:
# capture output information from the bluesky client and return it as a string
@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def update_until_complete(client):
    complete_output = ""
    empty_output_count = 0  # Track consecutive empty outputs

    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()

        # Check if the current output is empty
        if not new_output.strip():
            empty_output_count += 1  # Increment counter for empty outputs
        else:
            empty_output_count = 0  # Reset counter if output is not empty
            complete_output += new_output  # Add non-empty output to complete output

        # If there are two consecutive empty outputs, break the loop
        if empty_output_count >= 2:
            break

    # It's assumed you want to keep the last update outside the loop
    client.update()

    return complete_output

In [ ]:
# connect to client
client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()
client.update()

In [ ]:
with open('../prompts/basecmds.txt', 'r') as file:
    base_cmds = file.read()

Creating tools for agent

In [ ]:

# get all aircraft info
# get conflict information

@tool
def GetAllAircraftInfo(command: str = 'GETACIDS'):
    """Get each aircraft information at current time: position, heading (deg), track (deg), altitude, V/S (vertical speed), calibrated, true and ground speed and mach number. Input is 'GETACIDS'.
    
    Parameters:
    - command: str (default 'GETACIDS')
    
    Example usage:
    - GetAllAircraftInfo('GETACIDS')
    
    Returns:
    - str: all aircraft information
    """
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    print(f'LLM input:{command}')
    
    client.send_event(b'STACK', command)
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output

@tool
def GetConflictInfo(commad: str = 'SHOWTCPA'):
    """Use this tool to identify and get vital information on aircraft pairs in conflict. It gives you Time to Closest Point of Approach (TCPA), Quadrantal Direction (QDR), separation distance, Closest Point of Approach distance (DCPA), and Time of Loss of Separation (tLOS).
    
    Parameters:
    - command: str (default 'SHOWTCPA')
    
    Example usage:
    - GetConflictInfo('SHOWTCPA')
    
    Returns:
    - str: conflict information between aircraft pairs
    """
    client.send_event(b'STACK', 'SHOWTCPA')
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output


@tool
def ContinueMonitoring(time: str = '5'):
    """Monitor for conflicts between aircraft pairs for a specified time. 
    Parameters:
    - time (str): The time in seconds to monitor for conflicts. Default is 5 seconds.
    
    Example usage:
    - ContinueMonitoring('5')
    
    Returns:
    - str: The conflict information between aircraft pairs throughout the monitoring period.
    """
    for i in range(int(time)):
        client.send_event(b'STACK', 'SHOWTCPA')
        time.sleep(1)
        sim_output += str(i) + ' sec: \n' + update_until_complete(client) + '\n'
    return sim_output


@tool
def SendCommand(command: str):
    """
    Sends a command with optional arguments to the simulator and returns the output. 
    You can only send 1 command at a time.
    
    Parameters:
    - command (str): The command to send to the simulator. Can only be a single command, with no AND or OR operators.
    
    Example usage:
    - SendCommand('COMMAND_NAME ARG1 ARG2 ARG3 ... ARGn) # this command requires arguments
    - SendCommand('COMMAND_NAME') # this command does not require arguments
    
    Returns:
    str: The output from the simulator.
    """
    # Convert the command and its arguments into a string to be sent
    #command_with_args = ' '.join([command] + [str(arg) for arg in args])
    # Send the command to the simulator
    # client.update()  # Uncomment this if you need to update the client state before sending the command
    print(command)
    # replace " or ' in the command string with nothing
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    client.send_event(b'STACK', command)  
    # wait 1 second
    time.sleep(1)
    # Wait for and retrieve the output from the simulator
    sim_output = update_until_complete(client)
    if sim_output == '':
        return 'Command executed successfully.'
    if 'Unknown command' in sim_output:
        return sim_output + '\n' + 'Please use a tool QueryDatabase to search for the correct command.'
    return sim_output



@tool
def QueryDatabase(input: str):
    """If you want to send command to a simulator please first search which command you should use. For example if you want to create an aircraft, search for 'how do I create an aircraft'.
    Parameters:
    - input: str (the query to search for)
    Returns:
    - list: the top 5 results from the database
    """

    query_results = collection.query(
        query_texts=[input],
        n_results=5
    )

    return "The HDG command sets the aircraft's heading, disengaging LNAV mode. Use it by specifying the aircraft ID and desired heading in degrees, like: HDG acid, hdg_degrees. The aircraft ID is the unique identifier of the aircraft you want to control, and the heading is the desired direction in degrees. For example, to set the heading of aircraft ABC to 90 degrees, you would use the command: HDG ABC, 90 \n\n\n The ALT command adjusts an aircraft's altitude via autopilot, optionally setting vertical speed. Specify the aircraft ID, desired altitude in feet, and optionally, climb/descent speed in feet per minute, like ALT acid, alt, vspd. For example if you want to change aircraft KL123 height to 20000 ft the command is: ALT KL123 20000. or ALT KL123 FL200, or ALT KL123 FL200 10"
    #return query_results['documents'][0]


tools = [GetAllAircraftInfo, GetConflictInfo,
         SendCommand, QueryDatabase]

In [ ]:
input = "check if there are any conflicts between aircraft and if there are resolve them by changing altitude or heading. Your Goal is to have no conflicts between aircraft."

In [ ]:
# Example:
# Question: check if there are any conflicts between aircraft and if there are resolve them by changing altitude or heading. If there is no conflict then you don't need to do anything.
# Thought: I should check if there are any conflicts between aircraft with the GetConflictInfo tool.
# Action: GetConflictInfo
# Action Input: SHOWTCPA
# Observation: Conflict info ...
# Thought: There is a conflict between these aircraft .... Let me check their information
# Action: GetAllAircraftInfo
# Action Input: GETACIDS
# Observation: Aircraft info ...
# Thought: Based on ... I should ... because ... to resolve the conflict  
# Action: SendCommand
# Action Input: COMMAND_NAME ARG1 ARG2 ARG3 ...
# Observation: Command output ... I must confirm that the conflict is resolved
# Action: GetConflictInfo
# Action Input: SHOWTCPA
# Observation: Conflict info ...
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question


In [ ]:
# Get the prompt to use - you can modify this!
from langchain import hub
react_prompt = hub.pull("hwchase17/react")
react_prompt.template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, specify exactly one of the following tools [{tool_names}]. No additional text or comments permitted.
Action Input: the input argument to the action.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question




Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
react_prompt.pretty_print()



 Agent

In [ ]:
# Get the prompt to use - you can modify this!
openai_function_prompt = hub.pull("hwchase17/openai-functions-agent")
openai_function_prompt.pretty_print()

In [ ]:
import time
from langchain import hub







from langchain.agents import AgentExecutor, create_react_agent, create_json_chat_agent, create_xml_agent, create_openai_functions_agent







from langchain_openai import OpenAI, ChatOpenAI
from langchain_community.chat_models import ChatOllama








from langchain_anthropic import AnthropicLLM, ChatAnthropic







from langchain_google_genai import GoogleGenerativeAI







# # anthropic llm







# llm = ChatAnthropic(







#     model='claude-3-opus-20240229')  # Choose the LLM to use







# llm = GoogleGenerativeAI(







#     model="gemini-pro", google_api_key=os.getenv("GEMINI_API_KEY"))
model_name = "mixtral:8x22b-instruct"
llm = Ollama(base_url="https://ollama.junzis.com", model=model_name)
#llm = ChatOpenAI()
def invoke_until_success(llm):
    while True:
        try:
            # Attempt to invoke the method
            llm.invoke('hi')
            print("Invocation successful.")
            break  # Exit the loop if invocation was successful
        except Exception as e:
            print("Invocation failed, trying again...")
            time.sleep(1)  # Wait for a short period before retrying

# Example usage
# invoke_until_success(llm)
invoke_until_success(llm)








# Construct the ReAct agent








react_agent = create_react_agent(llm, tools, react_prompt)


openai_function_agent = create_openai_functions_agent(llm, tools, openai_function_prompt)







# Create an agent executor by passing in the agent and tools








react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True)


openai_function_executor = AgentExecutor(
    agent=openai_function_agent, tools=tools, verbose=True, handle_parsing_errors=True)
#json_agent_executor = AgentExecutor(agent=json_agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
client.send_event(b'STACK', 'IC simple/conflicts/2ac/case1.scn')

In [ ]:
update_until_complete(client)

In [ ]:
from langchain_core.tracers.context import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    out = react_agent_executor.invoke({"input": input})

In [ ]:

from langchain.agents import AgentExecutor, create_openai_tools_agent, OpenAIFunctionsAgent, create_react_agent

from langchain.memory import ConversationBufferMemory

# prompt = hub.pull("hwchase17/openai-tools-agent")
# memory = ConversationBufferMemory(
#     memory_key="chat_history", return_messages=True)

# Choose the LLM that will drive the agent
# llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


chat_history = []

message = SystemMessage(
    content=(
        "You are very powerful air traffic controller, that can use commands to interact and control the airspace. "
        "You can use tools to get information about aircrafts - get_all_aircraft_info, conflicts - get_conflict_info, send commands to the simulator to interact with aircraft - send_command_to_simulator and get information and search for commands in the database - query_database. If you are going to use send_command_to_simulator tool you must first search for the command you should use in the database. Search: how can I ..."
    )
)

prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name="chat_history")],
)


# agent_kwargs = {
#     "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")],
# }

# # Create the memory
# memory = ConversationBufferMemory(memory_key="memory", return_messages=True)


# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True)

In [ ]:
user_input = "check if there are any conflicts between aircraft and if there are resolve them by changing altitude or heading"
while user_input != 'exit':
    if user_input != 'exit':
        # Invoke the agent and pass the current chat history
        update_until_complete(client)
        response = agent_executor.invoke(
            {"input": user_input, "chat_history": chat_history})
        human_message = HumanMessage(content=user_input)
        ai_message = AIMessage(content=response['output'])
        
        chat_history.append(human_message)
        chat_history.append(ai_message)
        print(AIMessage(content=response['output']))
        
        time.sleep(2)
        

In [ ]:
response['intermediate_steps'][-1][0].log

In [ ]:
agent_executor.invoke({'input': "what is my name?"})

In [ ]:
# gpt-4-0125-preview
# gpt-3.5-turbo-0125
# gpt-3.5-turbo-instruct




llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)




#output_parser = StrOutputParser()
# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful air taffic control operator, that can also use tools to answer questions and perform tasks",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm_with_tools = llm.bind_tools(tools)
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [ ]:
prompt_template = PromptTemplate.from_template(conflict_prompt)

In [ ]:
input = prompt_template.format()
print(input)

In [ ]:
client.update()


In [ ]:
out = agent_executor.invoke({"input": input})


In [ ]:
out["output"]